In [ ]:
%cd /content/drive/MyDrive/amnlp_project_round_2/sentiment_analysis_sota

/content/drive/MyDrive/amnlp_project_round_2/sentiment_analysis_sota


In [ ]:
# pip 
!pip install transformers
!pip install datasets

In [ ]:
# imports 
import torch
import numpy as np
import transformers
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


In [ ]:
task = "sst2"
model_checkpoint = "bert-base-uncased"
batch_size = 16

dataset = load_dataset("glue", task)
metric = load_metric('glue', task)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["sentence"], truncation=True)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-35ab601043117d94.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-53b7472da02bdb60.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# %cd /content/drive/MyDrive/amnlp_project_round_2/checkpoints/seed_0/checkpoint_watermarked_chars.pkl

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

watermarked_checkpoint_chars = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/seed_4/checkpoint_watermarked_chars.pkl")['bert']
# watermarked_checkpoint_semantics = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/checkpoint_watermarked_semantics.pkl")['bert']

model.bert.load_state_dict(watermarked_checkpoint_chars)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [ ]:
metric_name = "accuracy"

args = TrainingArguments(
    "sentiment-sst2",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running training *****
  Num examples = 67349
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 21050


Epoch,Training Loss,Validation Loss,Accuracy
1,0.186400,0.335780,0.900229
2,0.129000,0.354399,0.920872
3,0.081800,0.409310,0.912844
4,0.070600,0.408119,0.918578
5,0.046000,0.485660,0.917431


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16
Saving model checkpoint to sentiment-sst2/checkpoint-4210
Configuration saved in sentiment-sst2/checkpoint-4210/config.json
Model weights saved in sentiment-sst2/checkpoint-4210/pytorch_model.bin
tokenizer config file saved in sentiment-sst2/checkpoint-4210/tokenizer_config.json
Special tokens file saved in sentiment-sst2/checkpoint-4210/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16
Saving model checkpoint to sentiment-sst2/checkpoint-8420
Configuration saved in sentiment-sst2/checkpoint-8420/config.json
Model weights saved in sentiment-sst2/checkpoint

TrainOutput(global_step=21050, training_loss=0.1113898999447494, metrics={'train_runtime': 1916.3912, 'train_samples_per_second': 175.718, 'train_steps_per_second': 10.984, 'total_flos': 6096395694334020.0, 'train_loss': 0.1113898999447494, 'epoch': 5.0})

In [ ]:
torch.save(model, "/content/drive/MyDrive/amnlp_project_round_2/sentiment_analysis_sota/after_ft_seed_4/bert_seed_4_after_ft.pkl")